# Fluvial corridor workflow

## Install GDAL
For Windows 10, install GDAL with OSGEO4W

[link to installer](https://trac.osgeo.org/osgeo4w/wiki/OSGeo4W_fr)

Choose *Express Install* then GDAL.

In your user environement variables add C:\OSGeo4W\bin to PATH.

## Set input parameters
Create a config.ini file following the example below.

```
[workspace]

    workdir=./path/to/outputs
    srs=EPSG:2154

[DataSources] ; each datasource are referenced with parameters below

    bdalti = BDALTI ; DEM with bigger resolution used to fill no data in finer resolution DEM, tif extent
    rgealti = RGEALTI ; reference DEM
    sources = SOURCES_BDTOPO ; hydrographic network sources

[Tilesets] ; Two overlapping grids are required to tile the calculations. The overlay allows to correct the errors between the junctions of the tiles.
    default = 10K_TILESET ; if tileset is define as default the first grid is selected
    10k = 10K_TILESET ; first grid
    10kbis = 10KBIS_TILESET ; second overlapping grids

; 
; Describe Datasources below wich are reference above in [DataSources] and [Tilesets]
; 

[BDALTI]

    type=datasource
    data=elevation
    filename=./tutorials/dem_to_dgo/inputs/DEM.tif ; relative path to file
    resolution=25.0 ; DEM resolution

[RGEALTI]

    type=datasource
    data=elevation
    filename=./tutorials/dem_to_dgo/inputs/DEM_5M.tif ; relative path to file
    resolution=5.0 ; DEM resolution

[SOURCES_BDTOPO]

    type = datasource
    data = hydrography
    filename = ./tutorials/dem_to_dgo/inputs/sources.gpkg ; relative path to file

[10K_TILESET]

    type=tileset
    index=./tutorials/dem_to_dgo/inputs/10k_tileset.gpkg ; relative path to file
    height=2000
    width=2000
    tiledir=10K
    resolution=10000.0

[10KBIS_TILESET]

    type=tileset
    index=./tutorials/dem_to_dgo/inputs/10kbis_tileset.gpkg ; relative path to file
    height=2000
    width=2000
    tiledir=10KBIS
    resolution=10000.0
```

Create a .env text file near to this notebook with
```FCT_CONFIG=/path/to/config.ini```

The outputs folder and name are generated automatically.
They are defined in yml files in the config/datasets folder or by tiledir attributes in config.ini section.
Activate the virtual environement in shell from the python-fct folder.

In [2]:
# Set number of your computer core for processing parallelization
import os
core = os.cpu_count()

In [ ]:
# Create your tileset

from fct.cli import Tiles
Tiles.CreateTileset('rgealti', 10000.0)

In [ ]:
# Prepare the DEM tiles and VRT

from fct.cli import Tiles
Tiles.DatasourceToTiles('rgealti', '10k', 'dem', processes=core, overwrite=True)
Tiles.DatasourceToTiles('rgealti', '10kbis', 'dem', processes=core, overwrite=True)

Arguments :
- bdalti or rgealti = input define in config.ini
- 10k = input tile grid define in config.ini
- dem = output file define in config/datasets/drainage.yml
- processes=core depends on the number of cores in your computer

In [ ]:
from fct.tileio import buildvrt
buildvrt('10k', 'dem', overwrite=True)
buildvrt('10kbis', 'dem', overwrite=True)

Arguments :
- 10k = input tile grid define in config.ini
- dem = output file define in config/datasets/drainage.yml

If you have two different scales DEM, you can fill the precise one with the less precise
A faire en amont du lissage?

## Create a drainage networks
### DEM preparation

#### DEM smoothing

In [ ]:
from fct.drainage import PrepareDEM
# Prepare the smoothing parameter
params = PrepareDEM.SmoothingParameters()

smoothing parameters : 
- elevations = input
- output = output smoothed dem
- window = smoothing window

In [ ]:
# change window size (defaut = 5)
params.window=5

In [ ]:
# uncomment below to print other parameters
# help(params)

In [ ]:
# Mean filter for each tile of the tileset
PrepareDEM.MeanFilter(params, overwrite=True, processes=core, tileset='10k')
PrepareDEM.MeanFilter(params, overwrite=True, processes=core, tileset='10kbis')

In [ ]:
# uncomment below to see an example of the the output tile path
# params.output.tilename(row=1, col=1)

In [ ]:
# Create virtual rasters for tiles
from fct.tileio import buildvrt
buildvrt('10k', 'smoothed', overwrite=True)
buildvrt('10kbis', 'smoothed', overwrite=True)

#### Fill sink
Create a depressionless DEM form each tile before get flow direction

In [ ]:
# import module
from fct.drainage import DepressionFill
# set parameters
params = DepressionFill.Parameters()

DepressionFill parameters :

- _elevations = input
- _hydrography = Stream network derived from cartography draped on DEM **a quoi ça sert?? c'est dans la fonction BurnTile pour creuser le MNT (avec le paramètre offset) avec un réseau hydrographique si cette couche est disponible (sauf que l'emplacement du fichier shp à fournir dans le yml est dans les outputs et non dans les input). J'ai l'impression que ce n'est plus utilisé...**
- _filled = output filled DEM tiles define in yml. DEM tiles, depression filling procedure, first pass: individual tile processing
- _labels = output watershed label raster tiles define in yml. Watershed labels (depression filling procedure)
- _graph = output watershed graph define in yml. Cross-tile watershed connection graph (depression filling procedure)
- _spillover = output watershed spillover define in yml. Tile spillovers, z resolution of dem-watershed-graph (depression filling procedure)
- _resolved = output final filled DEM tiles define in yml. DEM tiles, depression filling procedure, second pass: cross-tile resolution based on spillover graph
- _offset = ? **Est utiliser par la fonction BurnTile pour modifier les donnes du MNT avec le réseau hydrographique, par défault la valeur est de -1, cependant si elle est égale à -1 le réseau est surélevé et non creusé...**
- _exterior_data = DEM edge value. Default 9000 to create wall to avoid flow to get out.

In [ ]:
# uncomment below to show default parameters
# help(params)

In [ ]:
# change default parameters
params.elevations = 'smoothed'
# params.exterior_data = 0.0

In [ ]:
# Identifies and numbers the watersheds and continuous areas of the same elevation, with filling in the holes
DepressionFill.LabelWatersheds(params, overwrite=True, processes=core, tileset='10k')
DepressionFill.LabelWatersheds(params, overwrite=True, processes=core, tileset='10kbis')

# Calculates the overflow graph between the different tiles
DepressionFill.ResolveWatershedSpillover(params, overwrite=True, tileset='10k')
DepressionFill.ResolveWatershedSpillover(params, overwrite=True, tileset='10kbis')

# Adjusts the elevation of tile edge depressions, (altitude differential with the overflow point)
DepressionFill.DispatchWatershedMinimumZ(params, overwrite=True, processes=core, tileset='10k')
DepressionFill.DispatchWatershedMinimumZ(params, overwrite=True, processes=core, tileset='10kbis')

In [ ]:
# Create virtual rasters for tiles
from fct.tileio import buildvrt
buildvrt('10k', 'dem-watershed-labels', overwrite=True)
buildvrt('10kbis', 'dem-watershed-labels', overwrite=True)

buildvrt('10k', 'dem-filled', overwrite=True)
buildvrt('10kbis', 'dem-filled', overwrite=True)

buildvrt('10k', 'dem-filled-resolved', overwrite=True)
buildvrt('10kbis', 'dem-filled-resolved', overwrite=True)

#### Resolve flats
Resolve flat areas between filled DEM tiles

In [ ]:
# import module
from fct.drainage import BorderFlats
# set parameters
params = BorderFlats.Parameters()

BorderFlats parameters :

- _labels = watersheds with labels create in fill sinks step.
- _resolved = filled DEM tiles create in fill sinks, depression filling procedure, second pass: cross-tile resolution based on spillover graph
- _flat_labels = Flat labels output raster (depression filling procedure)
- _flat_graph = Cross-tile flat output connection graph (depression filling procedure)
- _flat_spillover = Tile spillovers output, z resolution of dem-flat-graph (depression filling procedure)
- _output = BorderFlats output. Filled DEM tiles, depression filling procedure, second pass: cross-tile resolution based on spillover graph

In [ ]:
# uncomment below to show default parameters
# help(params)

In [ ]:
# Identify flat area in previous filled DEM tiles and fix flat border
BorderFlats.LabelBorderFlats(params=params, processes=core, tileset='10k') 
BorderFlats.LabelBorderFlats(params=params, processes=core, tileset='10kbis')

BorderFlats.ResolveFlatSpillover(params=params, tileset='10k')
BorderFlats.ResolveFlatSpillover(params=params, tileset='10kbis')

# Adjusts the elevation of tile edge depressions, and calculates the depression map (altitude differential with the overflow point)
BorderFlats.DispatchFlatMinimumZ(params=params, overwrite=True, processes=core, tileset='10k')
BorderFlats.DispatchFlatMinimumZ(params=params, overwrite=True, processes=core, tileset='10kbis')

In [ ]:
# Create virtual rasters for tiles
from fct.tileio import buildvrt
buildvrt('10k', 'dem-flat-labels', overwrite=True)
buildvrt('10kbis', 'dem-flat-labels', overwrite=True)

buildvrt('10k', 'dem-drainage-resolved', overwrite=True)
buildvrt('10kbis', 'dem-drainage-resolved', overwrite=True)

### Flow direction

In [ ]:
# import module
from fct.drainage import FlowDirection
# set parameters
params = FlowDirection.Parameters()

FlowDirection parameters :
- _exterior = exterior domain. not found in yml input, should be a shape file to indentify the area out of the DEM and avoid flow to go out of this mask (flow = -1). **utilisé?**
- _elevations = filled DEM tiles raster input from Resolve flats step.
- _flow = D8 Flow Direction output raster, derived from DEM

In [ ]:
# uncomment below to show default parameters
# help(params)

In [ ]:
# change default parameters
# deactivate this parameter, exterior shapefile not define
params.exterior = 'off'

In [ ]:
# Calculate flow direction
# Resolve flats drainage direction and calculate D8 flow direction from adjusted elevations.
FlowDirection.FlowDirection(params=params, overwrite=True, processes=core, tileset='10k')
FlowDirection.FlowDirection(params=params, overwrite=True, processes=core, tileset='10kbis')

In [ ]:
# Create virtual rasters for tiles
from fct.tileio import buildvrt
buildvrt('10k', 'flow', overwrite=True)
buildvrt('10kbis', 'flow', overwrite=True)

### Flow accumulation

In [ ]:
# import module
from fct.drainage import Accumulate
# set parameters
params = Accumulate.Parameters()

Flow accumulation parameters :
- _exterior_flow = shapefile to connect outlets flow. **Pas dans le yml ni dans config.ini...**
- _elevations = initial raster DEM input
- _flow = Flow Direction input raster from flow direction step
- _outlets = 

In [ ]:
# uncomment below to show default parameters
# help(params)

In [ ]:
# change default parameters
params.elevations = 'dem-drainage-resolved'

In [ ]:
# Find tile outlets,ie. pixels connecting to anoter tile according to flow direction.
Accumulate.Outlets(params=params, processes=core, tileset='10k')
Accumulate.Outlets(params=params, processes=core, tileset='10kbis')

# Aggregate ROW_COL_INLETS_ORIGIN.geojson files into one ROW_COL_INLETS.shp shapefile
Accumulate.AggregateOutlets(params, tileset='10k')
Accumulate.AggregateOutlets(params, tileset='10kbis')

# Accumulate areas across tiles and output per tile inlet shapefiles with contributing area flowing into tile.
Accumulate.InletAreas(params=params, tileset='10k')
Accumulate.InletAreas(params=params, tileset='10kbis')

# Calculate D8 flow direction tile
Accumulate.FlowAccumulation(params=params, overwrite=True, processes=core, tileset='10k') 
Accumulate.FlowAccumulation(params=params, overwrite=True, processes=core, tileset='10kbis')

In [ ]:
# Create virtual rasters for tiles
from fct.tileio import buildvrt
buildvrt('10k', 'acc', overwrite=True)
buildvrt('10kbis', 'acc', overwrite=True)

### Stream network from source

In [ ]:
# import module
from fct.drainage import StreamSources
# same parameters as flow accumulation

**Add parameters for StreamSources**

In [ ]:
StreamSources.InletSources(params, tileset='10k')
StreamSources.InletSources(params, tileset='10kbis')

StreamSources.StreamToFeatureFromSources(min_drainage=500, processes=core, tileset='10k')
StreamSources.StreamToFeatureFromSources(min_drainage=500, processes=core, tileset='10kbis')

StreamSources.AggregateStreamsFromSources(tileset='10k')
StreamSources.AggregateStreamsFromSources(tileset='10kbis')

In [ ]:
# Create virtual rasters for tiles
from fct.tileio import buildvrt
buildvrt('10k', 'drainage-raster-from-sources')
buildvrt('10kbis', 'drainage-raster-from-sources')

#### Fix no flow

In [ ]:
# Find NoFlow pixels from RHTS
from fct.drainage import FixNoFlow
params = FixNoFlow.Parameters()

In [ ]:
# uncomment below to show default parameters
help(params)

In [ ]:
# change default parameters
params.noflow = 'noflow'
params.fixed = 'noflow-targets'

In [ ]:
FixNoFlow.DrainageRaster(params=params, processes=core, tileset='10k')
FixNoFlow.DrainageRaster(params=params, processes=core, tileset='10kbis')

In [ ]:
# Create virtual rasters for tiles
from fct.tileio import buildvrt
buildvrt('10k', 'drainage-raster-from-sources')
buildvrt('10kbis', 'drainage-raster-from-sources')

In [ ]:
FixNoFlow.NoFlowPixels(params=params, processes=core, tileset='10k')
FixNoFlow.NoFlowPixels(params=params, processes=core, tileset='10kbis')

In [ ]:
FixNoFlow.AggregateNoFlowPixels(params, tileset='10k')
FixNoFlow.AggregateNoFlowPixels(params, tileset='10kbis')

In [ ]:
# Fix NoFlow (create TARGETS shapefile and fix Flow Direction data)
FixNoFlow.FixNoFlow(params, tileset1='10k', tileset2='10kbis', fix=True)